In [1]:
#!pip install opencv-python mediapipe

In [2]:
#!pip install --upgrade protobuf numpy opencv-python --user

In [3]:
#!pip install msvc-runtime

In [4]:
#!pip uninstall mediapipe

In [5]:
#!pip install pyserial

In [1]:
import serial
import time

In [2]:
arduino = serial.Serial('COM7', 9600)

In [3]:
import cv2
import mediapipe as mp
import numpy as np

In [4]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

In [5]:
def classify_posture(landmarks):
    # Check if landmarks are available
    if not landmarks:
        return 0  # No one is in front of the camera

    # Calculate relative positions of landmarks to classify posture
    shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y
    hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP].y
    ankle_y = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y

    # Simple rules for detecting posture
    if abs(shoulder_y - hip_y) > 0.25 and abs(hip_y - ankle_y) > 0.3:
        return 2  # Standing
    elif abs(shoulder_y - hip_y) > 0.1 and abs(hip_y - ankle_y) < 0.3:
        return 1  # Sitting
    else:
        return 0  # Not standing or sitting (could be lying down or posture is undetected)


def control_arduino(landmarks):
    # Check if landmarks are available
    if not landmarks:
        return arduino.write(b'0')  # No one is in front of the camera

    # Calculate relative positions of landmarks to classify posture
    shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y
    hip_y = landmarks[mp_pose.PoseLandmark.LEFT_HIP].y
    ankle_y = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y

    # Simple rules for detecting posture
    if abs(shoulder_y - hip_y) > 0.25 and abs(hip_y - ankle_y) > 0.3:
        return arduino.write(b'2')  # Standing
    elif abs(shoulder_y - hip_y) > 0.1 and abs(hip_y - ankle_y) < 0.3:
        return  arduino.write(b'1')  # Sitting
    else:
        return arduino.write(b'0')

# Initialize Video Capture
cap = cv2.VideoCapture(0)  # Use 0 for the default camera
print("Starting video capture. Press 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Check if any landmarks are detected
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Classify posture and control Arduino based on landmarks
        posture = classify_posture(landmarks)
        control_arduino(landmarks)

        # Draw pose landmarks on the image
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display the posture on the frame
        posture_text = {0: 'No one', 1: 'Sitting', 2: 'Standing'}
        cv2.putText(frame, f'Posture: {posture_text[posture]}', 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        arduino.write(b'0')
        # If no landmarks detected, show "No one"
        cv2.putText(frame, 'Posture: No one', 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    # Display the frame
    cv2.imshow('Human Activity Recognition', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break


Starting video capture. Press 'q' to quit.
